In [20]:
import imp
environment = imp.load_source('environment','../../smarty/environment.py')

### Initialize Each Environment type

* Atom
* Bond
* Angle
* Torsion
* Improper 


In [21]:
atomEnv = environment.AtomChemicalEnvironment()
bondEnv = environment.BondChemicalEnvironment()
angleEnv = environment.AngleChemicalEnvironment()
torsionEnv = environment.TorsionChemicalEnvironment()
impropEnv = environment.ImproperChemicalEnvironment()

EnvList = [atomEnv, bondEnv, angleEnv, torsionEnv, impropEnv]
names = ['atom','bond','angle','torsion','improper']

for idx, Env in enumerate(EnvList):
    print("%10s: %s" % (names[idx], Env.asSMIRKS()))

      atom: [*:1]
      bond: [*:1]~[*:2]
     angle: [*:1]~[*:2]~[*:3]
   torsion: [*:1]~[*:2]~[*:3]~[*:4]
  improper: [*:1]~[*:2](~[*:3])~[*:4]


### Add descriptions to atoms

You can add descriptive features to atoms, you can chose between

* `ORtypes` 
* `ANDtypes` 

Here we'll use torsionEnv as an example since it has more random bonds and atoms to chose between

In [22]:
# create new torsion:
torsionEnv = environment.TorsionChemicalEnvironment()

# Select and atom
pickAtom = torsionEnv.selectAtom()
print "initially: ", torsionEnv.asSMIRKS()
print "chose atom", pickAtom.asSMIRKS()

pickAtom.addORtype('#1')
print torsionEnv.asSMIRKS(), "torsion after adding '#1' to atom ORtypes" 

pickAtom.addANDtype('H0')
print torsionEnv.asSMIRKS(), "torsion after adding 'H0' to atom ANDtypes"

initially:  [*:1]~[*:2]~[*:3]~[*:4]
chose atom [*:3]
[*:1]~[*:2]~[#1:3]~[*:4] torsion after adding '#1' to atom ORtypes
[*:1]~[*:2]~[#1;H0:3]~[*:4] torsion after adding 'H0' to atom ANDtypes


### Add same descriptions to bonds

Bonds have the same properties
bonds however are defined by the atoms they connect so when selecting a bond you get three items

* atom1 (atom on one end of bond)
* atom2 (atom on the otehr end of the bond)
* bond (bond object)


In [23]:
# create new torsion
torsionEnv = environment.TorsionChemicalEnvironment()

# pick a bond
atom1, atom2, pickBond = torsionEnv.selectBond()

print "initially: ", torsionEnv.asSMIRKS()
print "chose bond connecting %s %s %s" % (atom1.asSMIRKS(), pickBond.asSMARTS(), atom2.asSMIRKS())

pickBond.addORtype('-')
print torsionEnv.asSMIRKS(), "torsion after adding '-' to bond ORtypes" 

pickBond.addANDtype('@')
print torsionEnv.asSMIRKS(), "torsion after adding '@' to bond ANDtypes"
# Note now the bond between 1 and 2 is - instead of ~

initially:  [*:1]~[*:2]~[*:3]~[*:4]
chose bond connecting [*:2] ~ [*:1]
[*:1]-[*:2]~[*:3]~[*:4] torsion after adding '-' to bond ORtypes
[*:1]-;@[*:2]~[*:3]~[*:4] torsion after adding '@' to bond ANDtypes


### Add a new atom 

Specify an atom to attach a new atom. 
You can provide descriptors 
ORtypes and ANDtypes (AND'd to the end of the atoms' SMARTS string) 
for the atom and/or bond or leave them empty. 

The ORtypes and ANDtypes should be lists of strings, 
either way they will be used to create the new Atom and new Bond objects.

The `addAtom` method returns the created atom.

In [24]:
# create new torsion
torsionEnv = environment.TorsionChemicalEnvironment()

bondToAtom = torsionEnv.selectAtom()
print("Bond to atom %s" % bondToAtom.asSMIRKS())

atomORtypes = ['#6']
atomANDtypes = ['a']
bondORtypes = [':']
bondANDtypes = None

# addAtom(self, bondToAtom, bondORtypes=None, bondANDtypes=None, newORtypes=None, newANDtypes=None, newAtomIndex=None)
newAtom = torsionEnv.addAtom(bondToAtom, bondORtypes, bondANDtypes, atomORtypes, atomANDtypes, None)
print("New atom (%s) in torsion %s" % (newAtom.asSMIRKS(), torsionEnv.asSMIRKS()))

Bond to atom [*:1]
New atom ([#6;a]) in torsion [*:1](:[#6;a])~[*:2]~[*:3]~[*:4]


### Remove an atom

Removing an atom removes the specified atom if it meets the conditions. There are three possible outcomes. 

* The specified atom is a labeled atom so it cannot be removed - Returns `False`
* The specified atom connectes two other atoms so it cannot be removed - Returns `False`
* The specified atom is removed - Returns `True`

Below are examples for each of these scenarios with a Bond type environment

In [25]:
bondEnv = environment.BondChemicalEnvironment()
# atom1 is the first first atom in the bond
bondEnv.removeAtom(bondEnv.atom1)

Cannot remove labeled atom [*:1]


False

In [26]:
bondEnv = environment.BondChemicalEnvironment()
newAtom1 = bondEnv.addAtom(bondEnv.atom1, None, None, ['#6'], None)
newAtom2 = bondEnv.addAtom(newAtom1, None, None, ['#8'], None)
print bondEnv.asSMIRKS()

# Try removing atom 1
bondEnv.removeAtom(newAtom1)

# Remove atom 2, show new SMIRKS
bondEnv.removeAtom(newAtom2)
print bondEnv.asSMIRKS()

[*:1](~[#6]~[#8])~[*:2]
Cannot remove atom [#6] because it connects two atoms
[*:1](~[#6])~[*:2]


### AsSMARTS

This will be especially helpful for comparing this tool with our string manipulations for smarty with atom types
It returns the SMARTS string for atom1 as a single atom. 
This will allow us to use the same comparisons we are using now in testing the chemical environment tool


In [27]:
# create atom
atomEnv = environment.AtomChemicalEnvironment([['#1'],None])
print atomEnv.asSMARTS()
carbon = atomEnv.addAtom(atomEnv.atom1, '-', None, ['#6'])
print atomEnv.asSMARTS()
oxygen1 = atomEnv.addAtom(carbon, '-', None, ['#8'])
oxygen2 = atomEnv.addAtom(carbon, '-', None, ['#8'])
oxygen3 = atomEnv.addAtom(carbon, '-', None, ['#8'])
print atomEnv.asSMARTS()

[#1]
[#1$(*-[#6])]
[#1$(*-[#6](-[#8])(-[#8])-[#8])]


### Initialize a torsion with decorated atoms

Here is the documentation for initializing a torsion:
```
__init__(self, Atom1Info=[None, None], Bond1Info=[None, None], Atom2Info=[None, None], Bond2Info=[None, None], Atom3Info=[None, None], Bond3Info=[None, None], Atom4Info=[None, None])
    Initialize a chemical environment corresponding to matching four atoms (torsion).

    Parameters
    -----------
    Atom1Info, Atom2Info, Atom3Info, Atom4Info: list of sets, optional
        Comes in the form [AtomORtypes, AtomANDtypes]
        AtomORtypes: descriptors for the first atom that are connected with logical operation OR
        AtomANDtypes: descriptors for the first atom that are connected with the logical operation AND
    Bond1Info and Bond2Info, Bond3Info: list of sets, optional
        In the form [BondORtypes, BondANDtypes] similar to atom information
     
    For example:
        # Create a torsion centered around two tetravalent carbons with single ring bonds
        CarbonInfo = [['#6'], ['X4']
        BondInfo = [['-'], ['@']]
        torsion = TorsionChemicalEnvironment(Atom2Info = CarbonInfo, Bond2Info = BondInfo, Atom3Info = CarbonInfo)
        print torsion.asSMIRKS()
        f# "[*:1]~[#6X4:2]-;@[#6X4:3]~[*:4]"
```

You can think of this as the order of atoms and bonds in a torsion 

Atom1 -(bond1)- Atom2 -(bond2)- Atom3 -(bond3)- Atom4

In this case I will make the torsion centered on a carbon/carbon double bond with a hydrogen on one end and a tetravalent carbon on the other. 


In [30]:
# define atoms for torsion
hydrogen = [['#1'],None]
triC = [['#6'], ['X3']]
tetraC = [['#6'], ['X4']]

#define bonds for torsion
singlebond = [['-'],None]
doublebond = [['='], None]

# initiate torsion
torsion = environment.TorsionChemicalEnvironment(hydrogen, singlebond, triC, doublebond, triC, singlebond, tetraC)
